Apr 15, 2019 - smouradi - Code to convert the .bin files provided by Sandia to something more bemsolver-like

In [1]:
import numpy as np

In [82]:
fname = 'RS1096_12400_1.bin'

#from table 5 of HOA manual
electrodeNames = ['GND','RF']+['G'+str(i+1).zfill(2) for i in range(8)]+ \
                            ['L'+str(i+1).zfill(2) for i in range(16)]+ \
                            ['Q'+str(i+1).zfill(2) for i in range(40)]+ \
                            ['T'+str(i+1).zfill(2) for i in range(6)]+ \
                            ['Y'+str(i+1).zfill(2) for i in range(24)]
                
#from table 4 of HOA manual    
with open(fname,'rb') as f:
    
    #header
    raw = f.read(160)
    nElec = np.frombuffer(raw[8:16],np.int64)[0]
    nXYZ = np.frombuffer(raw[16:40],np.int64)
    vSets = np.frombuffer(raw[40:48],np.int64)[0]
    stepSize = np.frombuffer(raw[96:120])*1e3 #step size in mm (=5um)
    origin = np.frombuffer(raw[120:144])*1e3
    roi = np.array([np.array([0,nXYZ[i]-1])*stepSize[i]+origin[i] for i in range(3)])
    
    #electrode mapping
    raw = f.read(8*nElec)
    electrodes = np.frombuffer(raw,np.int64)
    
    #data
    raw = f.read()
    data = np.frombuffer(raw)
    dt = data.reshape(nElec,nXYZ[0]*nXYZ[1]*nXYZ[2])
    print len(dt)
    print len(dt[1])
    print stepSize


96
183495
[0.005 0.005 0.005]


In [89]:
#reshape into what is needed by old python multipole expansion code

gridXYZ = [np.linspace(roi[l,0],roi[l,1],nXYZ[l]) for l in range(3)]
fullCoords = [[gridXYZ[0][i], gridXYZ[1][j], gridXYZ[2][k]] \
            for i in range(len(gridXYZ[0])) for j in range(len(gridXYZ[1])) for k in range(len(gridXYZ[2]))]
print len(fullCoords)

### save all data from this .bin file
### CAUTION THIS TAKES A LONG TIME
#fullCoords = fullCoords*nElec
#fullData = [fullCoords[i]+[data[i]] for i in range(len(data))]
#np.savetxt('RS1096_12400_1_full.txt',fullData, delimiter=',')

### save effect of only Q electrodes
#electrodes = [0,1]+range(26,66)
#fullCoords = fullCoords*len(electrodes)
#fullData = [fullCoords[i]+[data[i]] for i in electrodes]
#np.savetxt('RS1096_12400_1_Qonly.txt',fullData,delimiter=',')

### save effect of only 28 electrodes initially controlled
electrodes = [1]+range(32,58)+[64,65]
#fullCoords = fullCoords*len(electrodes)

fullData = [gridXYZ[0],gridXYZ[1],gridXYZ[2],[]]

for i in electrodes:
    fullData[3] = np.append(fullData[3],dt[i])

np.savetxt('RS1096_12400_1_28onlyX.txt',fullData[0],delimiter=',')
np.savetxt('RS1096_12400_1_28onlyY.txt',fullData[1],delimiter=',')
np.savetxt('RS1096_12400_1_28onlyZ.txt',fullData[2],delimiter=',')
np.savetxt('RS1096_12400_1_28onlyV.txt',fullData[3],delimiter=',')



183495


In [86]:
len(fullData[3])

5321355